# Predictor of housing prices in Boston, MA in 1970s

#### Dependencies:

In [1]:
import numpy as np
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint #save model's parameters after each epoch
import os #to help save after each epoch


#optimization: use tensorboard to automatically identify lowest cost epoch
from tensorflow.keras.callbacks import TensorBoard


#### Load the data

In [2]:
(x_train, y_train), (x_valid, y_valid) = boston_housing.load_data()

In [3]:
x_train.shape #404 data points, each data point is array with size 13 
# 13 independent vars collected to determine housing price (various statistics around area)

(404, 13)

In [4]:
y_train.shape 

(404,)

In [5]:
x_valid.shape

(102, 13)

In [6]:
x_train[0] # 13 independent vars collected to determine housing price (various statistics around area)

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [7]:
y_train[0]  #the actual price of the house (in thousands of dollars)

15.2

#### Design neural network

In [8]:
model = Sequential()

# since we only have 13 inputs, a shallow architecture would suffice

#first hidden layer
model.add(Input(shape=(13,)))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())

#second hidden layer
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2)) #add dropout for last layer to prevent overfitting

#output layer
model.add(Dense(1, activation='linear')) #we are not passing outputs through any activation filter
#just want the raw numeric value (hence linear activation is equivalent of no activation function applied)


#output layer should have 1 neuron for regression-type models! 


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,185 (4.63 KB)

 Trainable params: 1,089 (4.25 KB)

 Non-trainable params: 96 (384.00 B)

#### Configure the model

In [10]:
model.compile(loss='mean_squared_error', optimizer='adam') #MSE actually faster than cross-entropy for regression

<b>Note:</b> the "accuracy" metric is excluded, because it is meaningless for regression! <u>We don't care</u> that the value predicted doesn't match exactly with the expected, just want to see if it is a good predictor overall 

In [11]:
output_dir = 'model_output/' #save our model weights after each epoch in training

In [12]:
run_name = 'regression_drop_20'
output_path = output_dir + run_name #subdirectory "regression_baseline" for subsequent modification results

In [13]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [14]:
model_checkpoint = ModelCheckpoint(output_path + '/{epoch:02d}.weights.h5', save_weights_only=True)


# save in the regression_baseline folder as hdf5 file format
# save_weights_only is referring to both weights/biases in the model


In [15]:
#Optimization with tensorboard:

tensorboard = TensorBoard(log_dir='logs/'+run_name) #new folder to store tensorboard for model wieghts

#### Training data

In [16]:
model.fit(x_train, y_train, 
         batch_size=8, epochs=32, verbose=1,
         validation_data=(x_valid, y_valid), 
         callbacks=[model_checkpoint, tensorboard])

Epoch 1/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 559.4454 - val_loss: 582.9285
Epoch 2/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 553.7529 - val_loss: 532.0159
Epoch 3/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 526.3248 - val_loss: 542.7103
Epoch 4/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 525.9873 - val_loss: 500.9003
Epoch 5/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 510.5572 - val_loss: 462.8831
Epoch 6/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - loss: 458.8572 - val_loss: 388.6208
Epoch 7/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - loss: 439.5242 - val_loss: 347.2101
Epoch 8/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 383.2845 - val_loss: 346.6067
Epoch 9/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 386.8966 - val_loss: 355.0493
Epoch 10/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 291.9877 - val_loss: 218.3517
Epoch 11/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 269.4744 - val_loss: 294.6658
Epoch 12/32
51/51 ━━━━━━━━

Seems like epoch 29 produced the lowest validation loss (val_loss: 28.4775). We can load the weights/biases from that epoch into our model

In [17]:
model.load_weights(output_path + '/29.weights.h5')

In [18]:
x_valid[42]

array([  9.32909,   0.     ,  18.1    ,   0.     ,   0.713  ,   6.185  ,
        98.7    ,   2.2616 ,  24.     , 666.     ,  20.2    , 396.9    ,
        18.13   ])

In [19]:
y_valid[42]

14.1

In [20]:
model.predict(np.reshape(x_valid[42], [1, 13])) #actual prediction for given data (feeds all 13 input vals into network)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


array([[15.034498]], dtype=float32)

Expected output was 14.1, model predicted 15.4443. Very close prediction. 

## Optimization: use tensorboard so that you don't have to manually scan for lowest validation loss